# Capstone Project - Short Answer Grading

In [1]:
# source code for SKLearn custom train.py
!pygmentize source/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd
from source.utils import generate_data

from sklearn.externals import joblib
from keras.layers.core import Dropout
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")

    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")

    return model


if __name__ == '__main__':
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job

    # Here we set up an argument parser to easily access

# Setup AWS

In [3]:
import pandas as pd
import boto3
import sagemaker

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [6]:
# should be the name of directory you created to save your features data
data_dir = 'data/seb'

# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/short_answer'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

# Training Basic Model

In [10]:
from sagemaker.sklearn.estimator import SKLearn
# instantiate a pytorch estimator
# estimator = SKLearn(entry_point='train.py',
#                     source_dir='source', # this should be just "source" for your code
#                     role=role,
#                     train_instance_count=1,
#                     train_instance_type='ml.c4.xlarge',
#                     hyperparameters = {
#                         'epochs': 200,
#                         'embedding_size': 30,
#                         'lstm_size': 100,
#                         'dropout': 0.2,
#                         'optimizer': 'adam'
#                     }
#                    )

In [17]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='source/train.py', 
                       role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       framework_version='1.12', 
                       py_version='py3',
                       script_mode=True,
                       hyperparameters = {
                        'epochs': 200,
                        'embedding_size': 30,
                        'lstm_size': 100,
                        'dropout': 0.2,
                        'optimizer': 'adam'
                    })

In [34]:
from tensorflow.python.training.adam import AdamOptimizer

In [45]:
%%time
# Train estimator on S3 training data
estimator.fit(input_data)

2019-06-24 21:36:44 Starting - Starting the training job...
2019-06-24 21:36:48 Starting - Launching requested ML instances......
2019-06-24 21:37:55 Starting - Preparing the instances for training......
2019-06-24 21:38:57 Downloading - Downloading input data...
2019-06-24 21:39:30 Training - Training image download completed. Training in progress.
2019-06-24 21:39:34,282 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-06-24 21:39:34,290 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-24 21:39:34,661 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-24 21:39:34,677 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-24 21:39:34,688 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "curren

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

--------------------------------------------------------------------------------------------------------------------

In [ ]:
import numpy as np
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # rounding and squeezing array
    test_preds = np.squeeze(predictor.predict(test_features))
    # Normalized to range 0 to 1 if needed
    if test_preds.max() > 1:
        test_preds = test_preds * (1 / test_preds.max())
    test_preds = np.round(test_preds)    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # print metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return tp, fp, fn, tn, precision, recall, accuracy

In [ ]:
test_data = pd.read_csv(os.path.join(training_dir, "test.csv"), header=None, names=None)


In [ ]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)
#normalize if needed
if test_y_preds.max() > 1:
    test_y_preds = test_y_preds * (1 / test_y_preds.max())
    

In [ ]:
# Second: calculate the test accuracy
tp, fp, fn, tn, precision, recall, accuracy = evaluate(predictor, test_x, test_y)

print(accuracy)

## print out the array of predicted and true labels, if you want
results = pd.concat([pd.DataFrame(test_y_preds), pd.DataFrame(np.round(test_y_preds)), pd.DataFrame(test_y.values)], axis=1)
results.columns = ['raw_predicted','predicted','actual']
# print('\nPredicted class labels: ')
# print(test_y_preds)
# print('\nTrue class labels: ')
# print(test_y.values)
print(results)